# Настройка

In [11]:
import time
import docker
from psycopg import connect
from psycopg.rows import dict_row
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm

PG_IMG   = "pgvector/pgvector:pg16"
PG_NAME  = "pgv-demo"
PG_PORT  = 5431            # порт на хосте
PG_USER  = "admin"
PG_PASS  = "secret"
PG_DB    = "testdb"
EMB_DIM = 384

client = docker.from_env()

# если контейнер уже есть - удалим
try:
    client.containers.get(PG_NAME).remove(force=True)
except docker.errors.NotFound:
    pass

container = client.containers.run(
    PG_IMG,
    name=PG_NAME,
    environment={
        "POSTGRES_USER": PG_USER,
        "POSTGRES_PASSWORD": PG_PASS,
        "POSTGRES_DB": PG_DB
    },
    ports={"5432/tcp": PG_PORT},
    detach=True,
    remove=False
)

In [12]:
import nltk
import re

nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

fileids = movie_reviews.fileids()

def tidy(s: str) -> str:
    s = " ".join(s.split())                          # схлопнуть все виды пробелов/переводы строк
    s = re.sub(r"\s+([.,;:!?…])", r"\1", s)          # убрать пробелы ПЕРЕД . , ; : ! ? …
    s = re.sub(r"([(\[{«“])\s+", r"\1", s)           # убрать пробел ПОСЛЕ ( [ { « “
    s = re.sub(r"\s+([)\]}»”])", r"\1", s)           # убрать пробел ПЕРЕД ) ] } » ”
    return s.strip()[:500]

texts = [tidy(movie_reviews.raw(fid)) for fid in fileids]

texts = texts[:100]
print("Документов:", len(texts), "\nПример:", texts[2][:50].replace("\n"," ") + "…")

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/shcher/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Документов: 100 
Пример: it is movies like these that make a jaded movie vi…


In [13]:
container.exec_run(
    f'psql -h localhost -U {PG_USER} -d {PG_DB} '
    '-c "CREATE EXTENSION IF NOT EXISTS vector;"',
    user="postgres"
)
print("Расширения установлены")

Расширения установлены


In [15]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2") 
EMB_DIM = 384

DDL = f"""
DROP TABLE IF EXISTS docs;
CREATE TABLE docs (
  id   BIGSERIAL PRIMARY KEY,
  text TEXT NOT NULL,
  emb  vector({EMB_DIM}) NOT NULL
);
"""
with connect(host="localhost", port=PG_PORT, user=PG_USER,
             password=PG_PASS, dbname=PG_DB) as con:
    with con.cursor() as cur:
        cur.execute(DDL)
        con.commit()

def encode_texts(batch):
    X = model.encode(batch, batch_size=64, show_progress_bar=False,
                     normalize_embeddings=True)
    return X.astype(np.float32)

embs = []
B = 10
for i in tqdm(range(0, len(texts), B)):
    embs.append(encode_texts(texts[i:i+B]))
embs = np.vstack(embs)
embs.shape

rows = [(texts[i], embs[i].tolist()) for i in range(len(texts))]
with connect(host="localhost", port=PG_PORT, user=PG_USER,
             password=PG_PASS, dbname=PG_DB) as con:
    with con.cursor() as cur:
        cur.executemany(
            "INSERT INTO docs (text, emb) VALUES (%s, %s)",
            rows
        )
        con.commit()

100%|██████████| 10/10 [00:02<00:00,  4.57it/s]


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

MODEL_PROVIDER = os.getenv("MODEL_PROVIDER", "open")  # "open" | "deepseek"
COMMON_KW = dict(max_completion_tokens=6000, temperature=0.7, seed=42, top_p=0.9)

MODEL_PROVIDER = os.getenv("MODEL_PROVIDER", "open")  # "open" | "deepseek"
COMMON_KW = dict(max_completion_tokens=6000, temperature=0.7, seed=42, top_p=0.9)

if MODEL_PROVIDER == "open":
    llm = ChatOpenAI(
        model=os.getenv("OPEN_MODEL_NAME", "cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit"),
        base_url=os.getenv("OPEN_BASE_URL", "http://91.219.226.145:11666/v1"),
        api_key=os.getenv("API_KEY", "EMPTY"),
        **COMMON_KW
    )
    print("success")
elif MODEL_PROVIDER == "deepseek":
    llm = ChatOpenAI(
        model=os.getenv("DEEPSEEK_MODEL", "deepseek-chat"),
        base_url=os.getenv("DEEPSEEK_BASE_URL", "https://api.deepseek.com/v1"),
        api_key=os.getenv("DEEPSEEK_API_KEY", "sk-7c8d537e261348f2a64876ca6350ace5"),
        **COMMON_KW
    )
else:
    raise ValueError("MODEL_PROVIDER должен быть 'open' или 'deepseek'")

print("LLM init:", llm.invoke("\int x^3 dx = ").content)

client = MultiServerMCPClient({
    "texts_db": {
        "transport": "streamable_http",
        "url": "http://127.0.0.1:8000/mcp"        
    }
})


tools = await client.get_tools()
print("TOOL LIST:", [t.name for t in tools])

<>:32: SyntaxWarning: invalid escape sequence '\i'
/tmp/ipykernel_10793/174193842.py:32: SyntaxWarning: invalid escape sequence '\i'
  print("LLM init:", llm.invoke("\int x^3 dx = ").content)


success


[2025-12-14 15:04:26] INFO _client.py:1025: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 15:04:26] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:04:26] INFO streamable_http.py:134: Received session ID: 9d28ae03266c428c828d807514932c9e
[2025-12-14 15:04:26] INFO streamable_http.py:146: Negotiated protocol version: 2025-06-18


LLM init: To evaluate the integral:

$$
\int x^3 \, dx
$$

We use the **power rule for integration**, which states:

$$
\int x^n \, dx = \frac{x^{n+1}}{n+1} + C \quad \text{for } n \ne -1
$$

Here, $ n = 3 $, so:

$$
\int x^3 \, dx = \frac{x^{3+1}}{3+1} + C = \frac{x^4}{4} + C
$$

### Final Answer:
$$
\boxed{\frac{x^4}{4} + C}
$$


[2025-12-14 15:04:27] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"
[2025-12-14 15:04:27] INFO _client.py:1740: HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:04:27] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:04:27] INFO _client.py:1740: HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"


TOOL LIST: ['search_vector']


# Проверка LLM

In [26]:
import xgrammar as xgr

grammar = r"""
root ::= (think_block "\n")  (final_answer)

final_answer ::= key_relevant ws ":" ws value_choice  #yaml
key_relevant ::= "question_is_relevant"
value_choice ::= "yes" | "no"
ws ::= " " | "\t"
nl ::= "\n"

think_block ::= "<think>\nOk," allowed_char "\n</think>"
allowed_char ::= ([\na-zA-Z0-9!@#$%^&*()_+{}\\|;:'",.? \-<>])*

"""

compiled_grammar = xgr.Grammar.from_ebnf(grammar)
str(compiled_grammar)

'root ::= ((think_block "\\n" final_answer))\nfinal_answer ::= ((key_relevant ws ":" ws value_choice))\nkey_relevant ::= (("question_is_relevant"))\nvalue_choice ::= (("yes") | ("no"))\nws ::= ((" ") | ("\\t"))\nnl ::= (("\\n"))\nthink_block ::= (("<think>\\nOk," allowed_char "\\n</think>"))\nallowed_char ::= ((allowed_char_1))\nallowed_char_1 ::= ("" | ([\\na-zA-Z0-9!@#$%^&*()_+{}\\\\|;:\\\'\\",.\\? \\-<>] allowed_char_1))\n'

In [27]:
llm = ChatOpenAI(
    model=os.getenv("OPEN_MODEL_NAME", "cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit"),
    base_url=os.getenv("OPEN_BASE_URL", "http://91.219.226.145:11666/v1"),
    api_key=os.getenv("API_KEY", "EMPTY"),
    extra_body={"top_k": 20, "chat_template_kwargs": {"enable_thinking": False}, "guided_grammar": str(compiled_grammar)},
    **COMMON_KW
)

In [29]:
prompt = """
Относится ли вопрос "Зачем козе баян?" к теории струн?
"""
response = llm.invoke(prompt)
print(response.content)

prompt = """
Относится ли вопрос "Почему черная дыра - черная?" к физике?
"""
response = llm.invoke(prompt)
print(response.content)

prompt = """
Относится ли вопрос "Надо ботать?" к МФТИ?
"""
response = llm.invoke(prompt)
print(response.content)

[2025-12-14 15:03:07] INFO _client.py:1025: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"


<think>
Ok, let's think step by step.

The question "Why does a goat need a bayan?" (in Russian: "Zachem koze bayan?") is a well-known Russian-language joke or riddle. It's absurd, humorous, and plays on the unexpected pairing of a goat (an animal) with a bayan (a Russian button accordion, a musical instrument). The humor comes from the lack of logical connection between the two, and the punchline is often something like: "Because it can't play the guitar!" or just nonsense. It's a classic example of a non-sequitur joke, popular in Russian internet culture and stand-up comedy.

Now, the theory of strings (string theory) is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity by modeling elementary particles as one-dimensional "strings" rather than point particles. It involves advanced mathematics, extra dimensions, quantum gravity, and is one of the leading candidates for a theory of everything.

Is there any legitimate scientific or co

[2025-12-14 15:03:10] INFO _client.py:1025: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"


<think>
Ok, let me think about this step by step.

The question is: "Why is a black hole black?"

First, I need to understand what a black hole is. From physics, a black hole is a region in space where gravity is so strong that nothing, not even light, can escape from it. This is due to the extreme curvature of spacetime caused by a very dense mass, as predicted by Einstein's general theory of relativity.

Now, why is it called "black"? Because it doesn't emit or reflect any light. Since light cannot escape its event horizon, we cannot see it directly with electromagnetic radiation (like visible light, radio waves, etc.). So, visually, it appears black.

This explanation is fundamentally rooted in physics: general relativity, gravity, the nature of light, and spacetime. There's no metaphysical or philosophical interpretation needed here; it's a direct consequence of physical laws.

Therefore, the question "Why is a black hole black?" is not just related to physics; it is a physics ques

[2025-12-14 15:03:16] INFO _client.py:1025: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"


<think>
Ok, let me think about this step by step.

The question is: "Does the question 'Nado botat?' relate to MIPT?"

First, let's understand the terms:

- "Nado botat?" is Russian slang for "Do we need to study?" or "Should we study?" The word "botat" (botat') is a colloquial, somewhat humorous or ironic form of "uchitsya" (to study), often used by students in a self-deprecating or joking way. It's commonly heard in Russian student circles, especially in technical universities.

- MIPT (Moscow Institute of Physics and Technology) is one of Russia's most prestigious technical universities, known for its rigorous academic standards and intense workload. It's often joked about in Russian student culture that MIPT students are constantly studying, under pressure, and use phrases like "botat" as a way to cope or joke about the workload.

Now, is the phrase "Nado botat?" specifically associated with MIPT?

While it's not an official slogan or motto of MIPT, it's very much in line with the 

# Проверка агента

In [3]:
import xgrammar as xgr

grammar = r"""
root ::= (think_block "\n") (final_answer)

final_answer ::= key_relevant ws ":" ws value_choice
key_relevant ::= "question_is_relevant"
value_choice ::= "yes" | "no"
ws ::= " " | "\t"
nl ::= "\n"

think_block ::= "<think>\nOk," allowed_char "\n</think>"
allowed_char ::= ([\na-zA-Z0-9!@#$%^&*()_+{}\\|;:'",.? \-<>])*

"""

compiled_grammar = xgr.Grammar.from_ebnf(grammar)
str(compiled_grammar)

llm = ChatOpenAI(
    model=os.getenv("OPEN_MODEL_NAME", "cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit"),
    base_url=os.getenv("OPEN_BASE_URL", "http://91.219.226.145:11666/v1"),
    api_key=os.getenv("API_KEY", "EMPTY"),
    extra_body={"guided_grammar": str(compiled_grammar)},
    **COMMON_KW
)

In [ ]:
llm = ChatOpenAI(
    model=os.getenv("OPEN_MODEL_NAME", "cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit"),
    base_url=os.getenv("OPEN_BASE_URL", "http://91.219.226.145:11666/v1"),
    api_key=os.getenv("API_KEY", "EMPTY"),
    **COMMON_KW
)

In [ ]:
from langgraph.prebuilt import create_react_agent


agent = create_react_agent(
    llm,
    tools,
    debug=True
)

С грамматикой агент не может использовать tools??

In [8]:
sql_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": f"Используя search_vector опиши, посвящена ли база данных теории струн."}]}
)
print(sql_response["messages"].pop().content)

[values] {'messages': [HumanMessage(content='Используя search_vector опиши, посвящена ли база данных теории струн.', additional_kwargs={}, response_metadata={}, id='9cca68ff-8fe4-48b3-9117-b44c70847f5f')]}


[2025-12-14 15:07:10] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content="<think>\nOk, I need to determine whether the database is related to string theory. To do this, I'll use the search_vector function with a query about string theory to see if there are relevant results in the database.\n</think>\nquestion_is_relevant : yes", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 289, 'total_tokens': 351, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-ecc5c58ab91249359a03e33a7b25fec7', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--78dc2854-ab7b-40f0-9be8-45896c8d3318-0', usage_metadata={'input_tokens': 289, 'output_tokens': 62, 'total_tokens': 351, 'input_token_details': {}, 'output_token_details': {}})]}}
[values] {'messages': [HumanMessage(content='Используя search_vector опиши, посвящена л

# Вариант без строгой грамматики

In [9]:
llm = ChatOpenAI(
    model=os.getenv("OPEN_MODEL_NAME", "cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit"),
    base_url=os.getenv("OPEN_BASE_URL", "http://91.219.226.145:11666/v1"),
    api_key=os.getenv("API_KEY", "EMPTY"),
    **COMMON_KW
)

agent = create_react_agent(
    llm,
    tools,
    debug=True
)

In [11]:
sql_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": f"Используя search_vector опиши, посвящена ли база ТЕОРИИ СТРУН. Для ответа используй следующую грамматику: {grammar}"}]}
)
print(sql_response["messages"].pop().content)

[values] {'messages': [HumanMessage(content='Используя search_vector опиши, посвящена ли база ТЕОРИИ СТРУН. Для ответа используй следующую грамматику: \nroot ::= (think_block "\\n") (final_answer)\n\nfinal_answer ::= key_relevant ws ":" ws value_choice\nkey_relevant ::= "question_is_relevant"\nvalue_choice ::= "yes" | "no"\nws ::= " " | "\\t"\nnl ::= "\\n"\n\nthink_block ::= "<think>\\nOk," allowed_char "\\n</think>"\nallowed_char ::= ([\\na-zA-Z0-9!@#$%^&*()_+{}\\\\|;:\'",.? \\-<>])*\n\n', additional_kwargs={}, response_metadata={}, id='a557d061-9449-4255-9dd0-e3cb8cd9b88e')]}


[2025-12-14 15:07:44] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 15:07:44] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:44] INFO streamable_http.py:134: Received session ID: a397c2e16bac403b9b93e7346703a867
[2025-12-14 15:07:44] INFO streamable_http.py:146: Negotiated protocol version: 2025-06-18
[2025-12-14 15:07:44] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"
[2025-12-14 15:07:44] INFO _client.py:1740: HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:44] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:44] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:44] INFO _client.py:1740: HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-06d1589364b44c5a9051c54f8d48e76a', 'function': {'arguments': '{"query": "теория струн", "k": 5}', 'name': 'search_vector'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 410, 'total_tokens': 441, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-aa1916f9e0774e7abc73513c6c72d7b6', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ce05cfd3-879e-49e9-86f4-9a1c1c29b823-0', tool_calls=[{'name': 'search_vector', 'args': {'query': 'теория струн', 'k': 5}, 'id': 'chatcmpl-tool-06d1589364b44c5a9051c54f8d48e76a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 410, 'output_tokens': 31, 'total_tokens': 441, 'input_token_details': {}, 'output_token_details': {

[2025-12-14 15:07:45] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 15:07:45] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:45] INFO streamable_http.py:134: Received session ID: ab9e0be8911043f19000ee5cae05a175
[2025-12-14 15:07:45] INFO streamable_http.py:146: Negotiated protocol version: 2025-06-18
[2025-12-14 15:07:45] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"
[2025-12-14 15:07:45] INFO _client.py:1740: HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:45] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:45] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:45] INFO _client.py:1740: HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-793d933f6fbb44278df983a30dfc15b6', 'function': {'arguments': '{"query": "теория струн в физике", "k": 5}', 'name': 'search_vector'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 1233, 'total_tokens': 1269, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-0698822d019b4b019a105536f5f4b055', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6d0edf88-1f84-4cc1-aa7b-ee77a23b498f-0', tool_calls=[{'name': 'search_vector', 'args': {'query': 'теория струн в физике', 'k': 5}, 'id': 'chatcmpl-tool-793d933f6fbb44278df983a30dfc15b6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1233, 'output_tokens': 36, 'total_tokens': 1269, 'input_token_details': {}, 'ou

[2025-12-14 15:07:46] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 15:07:46] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:46] INFO streamable_http.py:134: Received session ID: 5340648a719a4db3bfa8631c3d469de8
[2025-12-14 15:07:46] INFO streamable_http.py:146: Negotiated protocol version: 2025-06-18
[2025-12-14 15:07:46] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"
[2025-12-14 15:07:46] INFO _client.py:1740: HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:46] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:46] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:07:46] INFO _client.py:1740: HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-3d2793f626994b24b1b4b659f3c30fae', 'function': {'arguments': '{"query": "теория струн объяснение", "k": 5}', 'name': 'search_vector'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 2089, 'total_tokens': 2123, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-40ab134d34d04f7aa6770a0027ae828e', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ba1624cd-c337-4afe-99d3-2fa5cb897d8e-0', tool_calls=[{'name': 'search_vector', 'args': {'query': 'теория струн объяснение', 'k': 5}, 'id': 'chatcmpl-tool-3d2793f626994b24b1b4b659f3c30fae', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2089, 'output_tokens': 34, 'total_tokens': 2123, 'input_token_details': {},

[2025-12-14 15:07:47] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content=' Ok, I have performed three semantic vector searches with queries related to "теория струн" (string theory) in different forms: "теория струн", "теория струн в физике", and "теория струн объяснение". The results returned snippets from a database of movie reviews and film-related text. None of the snippets mention string theory, theoretical physics, or any related scientific concepts. All results are about films, directors, actors, and plot summaries, with no relevance to theoretical physics or string theory. Therefore, the database does not appear to contain any content related to string theory. \n\nquestion_is_relevant : no', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 142, 'prompt_tokens': 2924, 'total_tokens': 3066, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatc

In [13]:
sql_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": f"Используя search_vector опиши, посвящена ли база ФИЛЬМАМ. Для ответа используй следующую грамматику: {grammar}"}]}
)
print(sql_response["messages"].pop().content)

[values] {'messages': [HumanMessage(content='Используя search_vector опиши, посвящена ли база ФИЛЬМАМ. Для ответа используй следующую грамматику: \nroot ::= (think_block "\\n") (final_answer)\n\nfinal_answer ::= key_relevant ws ":" ws value_choice\nkey_relevant ::= "question_is_relevant"\nvalue_choice ::= "yes" | "no"\nws ::= " " | "\\t"\nnl ::= "\\n"\n\nthink_block ::= "<think>\\nOk," allowed_char "\\n</think>"\nallowed_char ::= ([\\na-zA-Z0-9!@#$%^&*()_+{}\\\\|;:\'",.? \\-<>])*\n\n', additional_kwargs={}, response_metadata={}, id='df596146-f6b0-40de-ab87-6a152638e964')]}


[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:05] INFO streamable_http.py:134: Received session ID: 71f1d09cbc4e4383b3e23f4bf14e1c3c
[2025-12-14 15:08:05] INFO streamable_http.py:146: Negotiated protocol version: 2025-06-18
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-bd1d1c24ad0f45e08bc8230d6fe00e22', 'function': {'arguments': '{"query": "содержит ли база данных информацию о фильмах", "k": 5}', 'name': 'search_vector'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 405, 'total_tokens': 442, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-d4f7e980eedf41da86631d19178fc82a', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--479b582b-379e-49bd-922b-0f219b5a0b60-0', tool_calls=[{'name': 'search_vector', 'args': {'query': 'содержит ли база данных информацию о фильмах', 'k': 5}, 'id': 'chatcmpl-tool-bd1d1c24ad0f45e08bc8230d6fe00e22', 'type': 'tool_call'}], usage_metadata={'input_tokens': 405, 'output_tokens': 37, 'total_to

[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:05] INFO streamable_http.py:134: Received session ID: 16540dfafaf94e04bbb4cd711293168a
[2025-12-14 15:08:05] INFO streamable_http.py:146: Negotiated protocol version: 2025-06-18
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:05] INFO _client.py:1740: HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-5211e08564744bdfb9c120d27c615fe6', 'function': {'arguments': '{"query": "содержатся ли в базе данных рецензии на фильмы", "k": 5}', 'name': 'search_vector'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 1283, 'total_tokens': 1324, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-57afef10ecf24e1c92cde396658e66df', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a88f7ee9-c158-4825-b556-cad9b1baa541-0', tool_calls=[{'name': 'search_vector', 'args': {'query': 'содержатся ли в базе данных рецензии на фильмы', 'k': 5}, 'id': 'chatcmpl-tool-5211e08564744bdfb9c120d27c615fe6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1283, 'output_tokens': 41, 't

[2025-12-14 15:08:06] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 15:08:06] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:06] INFO streamable_http.py:134: Received session ID: 1cfa9e9b10f747e09fdb83112f12877c
[2025-12-14 15:08:06] INFO streamable_http.py:146: Negotiated protocol version: 2025-06-18
[2025-12-14 15:08:06] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"
[2025-12-14 15:08:06] INFO _client.py:1740: HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:06] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:06] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:06] INFO _client.py:1740: HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-06be52e2edd64586b0bf7c619f45356c', 'function': {'arguments': '{"query": "содержит ли база данных обзоры кинокартины", "k": 5}', 'name': 'search_vector'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 2136, 'total_tokens': 2177, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-717f837cf85b4eb6b1f5c51202cbfb99', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e3874efa-9212-4b93-b795-4403ddcd0c6c-0', tool_calls=[{'name': 'search_vector', 'args': {'query': 'содержит ли база данных обзоры кинокартины', 'k': 5}, 'id': 'chatcmpl-tool-06be52e2edd64586b0bf7c619f45356c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2136, 'output_tokens': 41, 'total_tok

[2025-12-14 15:08:08] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='Ok, в результате поиска по запросам, связанным с фильмами, рецензиями и обзорами кинокартин, в базе данных найдены несколько релевантных записей. В них упоминаются названия фильмов, режиссеры, актеры и содержатся оценки и комментарии к кинокартинах, например: "The English Patient", "The Limey", "In Praise of Love", "Shattered Image", "Wild Things", "Relentless" и другие. Это однозначно указывает на то, что база данных содержит информацию о фильмах.\n\nquestion_is_relevant : yes', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 3027, 'total_tokens': 3184, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-84ad48c52ecc41d88bec81f80d5b5119', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c6132523-78a3-41c1-9829-e82e270a478e

In [14]:
sql_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": f"Используя search_vector опиши, есть ли в БД что-то про РОМАНТИКУ Для ответа используй следующую грамматику: {grammar}"}]}
)
print(sql_response["messages"].pop().content)

[values] {'messages': [HumanMessage(content='Используя search_vector опиши, есть ли в БД что-то про РОМАНТИКУ Для ответа используй следующую грамматику: \nroot ::= (think_block "\\n") (final_answer)\n\nfinal_answer ::= key_relevant ws ":" ws value_choice\nkey_relevant ::= "question_is_relevant"\nvalue_choice ::= "yes" | "no"\nws ::= " " | "\\t"\nnl ::= "\\n"\n\nthink_block ::= "<think>\\nOk," allowed_char "\\n</think>"\nallowed_char ::= ([\\na-zA-Z0-9!@#$%^&*()_+{}\\\\|;:\'",.? \\-<>])*\n\n', additional_kwargs={}, response_metadata={}, id='46e9f1b2-6815-4d42-99ea-f1485fef2b06')]}


[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:28] INFO streamable_http.py:134: Received session ID: 2706dae4324d4a5cadcfc4a0443b3f6e
[2025-12-14 15:08:28] INFO streamable_http.py:146: Negotiated protocol version: 2025-06-18
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-53eb0cd590d94c21909d86a0e352cc49', 'function': {'arguments': '{"query": "РОМАНТИКА", "k": 5}', 'name': 'search_vector'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 407, 'total_tokens': 437, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-737cc34e0b0b48f79cbba6b573a60789', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--499ca96a-6f77-4125-85ec-0cd9206ffbc1-0', tool_calls=[{'name': 'search_vector', 'args': {'query': 'РОМАНТИКА', 'k': 5}, 'id': 'chatcmpl-tool-53eb0cd590d94c21909d86a0e352cc49', 'type': 'tool_call'}], usage_metadata={'input_tokens': 407, 'output_tokens': 30, 'total_tokens': 437, 'input_token_details': {}, 'output_token_details': {}})]}}

[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:28] INFO streamable_http.py:134: Received session ID: 9d2336d44b1749cc8d511c2c485e2ccb
[2025-12-14 15:08:28] INFO streamable_http.py:146: Negotiated protocol version: 2025-06-18
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:28] INFO _client.py:1740: HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-03234e46543146f6b12864c4c646a6a4', 'function': {'arguments': '{"query": "романтика", "k": 5}', 'name': 'search_vector'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 1262, 'total_tokens': 1290, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-2fce64de85714b8caa22b6e05d73f22a', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4ca9f3f4-ecf4-4a53-a50e-df36de848c6c-0', tool_calls=[{'name': 'search_vector', 'args': {'query': 'романтика', 'k': 5}, 'id': 'chatcmpl-tool-03234e46543146f6b12864c4c646a6a4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1262, 'output_tokens': 28, 'total_tokens': 1290, 'input_token_details': {}, 'output_token_details': {}}

[2025-12-14 15:08:29] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 15:08:29] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:29] INFO streamable_http.py:134: Received session ID: c8b403fd0b3a42d5877c7acc575cc65e
[2025-12-14 15:08:29] INFO streamable_http.py:146: Negotiated protocol version: 2025-06-18
[2025-12-14 15:08:29] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"
[2025-12-14 15:08:29] INFO _client.py:1740: HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:29] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:29] INFO _client.py:1740: HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"
[2025-12-14 15:08:29] INFO _client.py:1740: HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-5a41024465e74f78818a37fc1b5cedba', 'function': {'arguments': '{"query": "любовь", "k": 5}', 'name': 'search_vector'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 2115, 'total_tokens': 2143, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-92a8ed2e2a0642d8ac4ddd7597b3a7f5', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c919816c-905a-427c-863b-f3c298347816-0', tool_calls=[{'name': 'search_vector', 'args': {'query': 'любовь', 'k': 5}, 'id': 'chatcmpl-tool-5a41024465e74f78818a37fc1b5cedba', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2115, 'output_tokens': 28, 'total_tokens': 2143, 'input_token_details': {}, 'output_token_details': {}})]}}
[

[2025-12-14 15:08:31] INFO _client.py:1740: HTTP Request: POST http://91.219.226.145:11666/v1/chat/completions "HTTP/1.1 200 OK"


[updates] {'agent': {'messages': [AIMessage(content=' Ok, в базе данных есть упоминания о романтике и любви, но они не являются центральной темой. В результатах поиска по запросам "РОМАНТИКА", "романтика" и "любовь" присутствуют фрагменты, где упоминаются романтические отношения, например, в фильмах "La Femme Nikita", "The English Patient", "The Trumpet of the Swan". Однако это скорее побочные элементы сюжета, а не глубокое или прямое освещение темы романтики. Вывод: хотя тема частично затрагивается, она не представлена в явном, значимом виде. \nquestion_is_relevant : no', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 176, 'prompt_tokens': 2980, 'total_tokens': 3156, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'cpatonn/Qwen3-Next-80B-A3B-Instruct-AWQ-4bit', 'system_fingerprint': None, 'id': 'chatcmpl-e48305d7111e4e53966adb70e445ec4c', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id=